In this notebook we calculate approximate values of epsilon based on demographic data.

For China and Taiwan, the POLYMOD contact matrices are used. The average number of contacts for imported and local cases is calculated based on their age distributions. 

For Hong Kong, epsilon is estimated as the ratio of contacts in a graph of transmission events.

China:
Wu, Biao, et al. "Compare the epidemiological and clinical features of imported and local COVID-19 cases in Hainan, China." Infectious Diseases of Poverty 9.1 (2020): 1-11.

Taiwan:
Wu, Chun-Han, et al. "Epidemiological features of domestic and imported cases with COVID-19 between January 2020 and March 2021 in Taiwan." Medicine 100.39 (2021).

Hong Kong:
Liu, Yang, Zhonglei Gu, and Jiming Liu. "Uncovering transmission patterns of COVID-19 outbreaks: A region-wide comprehensive retrospective study in Hong Kong." EClinicalMedicine (2021): 100929.

In [1]:
# Import libraries
import numpy as np
import branchpro
import pandas as pd

# China

In [2]:
contact_matrix = pd.read_csv(
    '../data_library/polymod/china.txt',
    sep='\t',
    header=None)
    
# Get total contacts by age group
contact_matrix = contact_matrix.to_numpy()
total_contacts = np.sum(contact_matrix, axis=0)

# Get average contacts for age ranges of 10 years
c = [0.5 * (total_contacts[2*i] + total_contacts[2*i+1]) for
                       i in range(len(total_contacts)//2)]

# Average the first three entries to get contacts for age 0--29
c[0] = (c[0] + c[1] + c[2]) / 3
del c[1]
del c[2]

c = np.array(c)

# Data from the paper
local_cases = np.array([2, 10, 8, 9, 5, 1])
imported_cases = np.array([4, 10, 7, 16, 14, 5])

local_avg_conts = np.sum(local_cases * c) / np.sum(local_cases)
imported_avg_conts = np.sum(imported_cases * c) / np.sum(imported_cases)

estimate_epsilon = imported_avg_conts / local_avg_conts
print('China epsilon={}'.format(estimate_epsilon))

China epsilon=0.7856472812794634


# Taiwan

In [3]:
contact_matrix = pd.read_csv(
    '../data_library/polymod/taiwan.txt',
    sep='\t',
    header=None)
    
# Get total contacts by age group
contact_matrix = contact_matrix.to_numpy()
total_contacts = np.sum(contact_matrix, axis=0)

# Get average contacts for age ranges of 10 years
c = [0.5 * (total_contacts[2*i] + total_contacts[2*i+1]) for
                       i in range(len(total_contacts)//2)]

# Average the first two entries to get contacts for age 0--19 and 60+
c[0] = (c[0] + c[1]) / 2
del c[1]

c[-1] = (c[-1] + c[-2]) / 2
del c[-2]

c = np.array(c)

# Data from the paper
local_cases = np.array([5, 15, 14, 10, 18, 15])
imported_cases = np.array([45, 385, 250, 110, 79, 84])

local_avg_conts = np.sum(local_cases * c) / np.sum(local_cases)
imported_avg_conts = np.sum(imported_cases * c) / np.sum(imported_cases)

estimate_epsilon = imported_avg_conts / local_avg_conts
print('Taiwan epsilon={}'.format(estimate_epsilon))

Taiwan epsilon=1.23617385222926


# Hong Kong

In [4]:
# Mean outdegrees for each entry in the table
k = np.array([1, 3, 6, 12, 24, 48, 96])

# Data from the paper
linked_imp = np.array([37, 2, 2, 0, 0, 0, 0])
linked_local = np.array([3049, 614, 404, 268, 184, 96, 32])
local = np.array([1607, 425, 284, 94, 15, 24, 3])
linked_possible_local = np.array([38, 9, 11, 4, 0, 0, 0])
imported = np.array([1728, 65, 50, 5, 0, 0, 0])
possible_local = np.array([92, 9, 1, 0, 1, 0, 0])

# Assume that these are local cases
local = linked_imp + linked_local + local + linked_possible_local + possible_local

local_avg_outdegree = np.sum(local * k) / np.sum(local)
imported_avg_outdegree = np.sum(imported * k) / np.sum(imported)

estimate_epsilon = imported_avg_outdegree / local_avg_outdegree
print('Hong Kong epsilon={}'.format(estimate_epsilon))

Hong Kong epsilon=0.2956532926187952
